In [1]:
!pip install detecto

In [4]:
import torch
import numpy as np

from detecto import core, utils

In [7]:
# define the labels and load the trained model
labels = ['+', '-', '/', '*', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
model = core.Model.load('trained_model.pth', labels)

In [8]:
# read image
image = utils.read_image('JPEG_DETECTION_IMAGE.jpg')
predictions = model.predict(image)

labels, boxes, scores = predictions

In [9]:
# function checks if two rectangular are overlapped
def do_overlap(rect1, rect2):
    # If x-axis are overlapped
    if rect1[0] < rect2[0] < rect1[2] or rect1[0] < rect2[2] < rect1[2] or (rect2[0] < rect1[0] and rect1[2] < rect2[2]):
      # If y-axis are overlapped
        if rect1[1] < rect2[1] < rect1[3] or rect1[1] < rect2[3] < rect1[3] or (rect2[1] < rect1[1] and rect1[3] < rect2[3]):
          return True
    return False


# get updated tensors of boxes and labels
def remove_intersecting_boxes(boxes, labels):
  upd_boxes, upd_labels = torch.tensor([]), []
  upd_num = 0
  for i in range(len(boxes)):
    intersection = False
    for j in range(upd_num):
      if (do_overlap(upd_boxes[j], boxes[i])):
        intersection = True
        break
    if not intersection:
      upd_boxes = torch.cat((upd_boxes, boxes[i].view(1, -1)))
      upd_labels.append(labels[i])
      upd_num += 1
  return (upd_boxes, upd_labels)

In [9]:
# updated boxes and labels
ub, ul = remove_intersecting_boxes(boxes, labels)

In [19]:
# sort in order of x-axis value
x_axis = np.array([axis[0] for axis in ub])
ind_pos = np.argsort(x_axis)
sort_lab = ul[ind_pos]

# implement the final string
final_str = ''
for i in range(len(sort_lab)):
  final_str += sort_lab[i]

In [19]:
# write what we detect on image
file = open("result_str", "w")
file.write(final_str)
file.close()